In [ ]:
# Plot ad hoc CIFAR10 instances
# Simple CNN model for CIFAR-10
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adadelta, RMSprop
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt

import os
import time
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:

# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
batch_normalization = True
model = Sequential()

#Layer 1 
model.add( Conv2D(48, kernel_size=(3,3),strides=(1,1), activation='relu', padding='same', input_shape=X_train.shape[1:] ) )
model.add( MaxPooling2D(pool_size=(2,2),strides=(2,2)) )
if batch_normalization: model.add(BatchNormalization())
#Layer 2 
model.add( Conv2D(96, kernel_size=(3,3),strides=(2,2), activation='relu', padding='same') )
model.add( MaxPooling2D(pool_size=(2,2),strides=(2,2)) )

#Fully 
model.add(Flatten())
if batch_normalization: model.add(BatchNormalization()) 
model.add(Dense(512, activation='tanh'))
if batch_normalization: model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))

# Compile model
epochs = 10
batch_size = 32
sgd = SGD(lr=0.01, momentum = 0.9, decay=0.001 , nesterov=False)
# ada_delta = Adadelta(learning_rate=0.001, rho=0.9)
rms = RMSprop(learning_rate=0.001, rho=0.9)


model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=rms,
              metrics=['accuracy'])

model.summary()



In [ ]:
checkpoint_path = "1st.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Fit the model
t0=time.time()
model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,  callbacks=[cp_callback])
t1=time.time()

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

print("training time:", round(t1-t0, 3), "s") # the time would be round to 3 decimal in seconds


In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()